# Tutorial 2: Reinforcement Learning Challenges

This is intended as a very short preamble to the Q-value algorithm with tabular methods and following topics. If you don't have any notions in Reinforcement Learning (RL) we suggest you to read some introduction about it.

<Insert other sources here>

* Sutton, Barto - Reinforcement Learning: An Introduction, Chapter 1 (gentle introduction) <br> https://www.google.com/search?client=ubuntu&channel=fs&q=reinforcement+learning+an+introduction&ie=utf-8&oe=utf-8

* OpenAI Spinning Up introduction (cool stuff, a bit formal) <br>
    https://spinningup.openai.com/en/latest/spinningup/rl_intro.html
    
* Medium articles (we just link one of the simplest one, but it's really plenty of them, most of which are really valid) <br>
    https://towardsdatascience.com/reinforcement-learning-demystified-36c39c11ec14

* Our presentation for the final examination with a more in-depth introduction to RL: <br>
https://docs.google.com/presentation/d/1EJmlQ-L-AC-snDwLroGnENuX4wGkJSlqsndbm1H76gA/edit?usp=sharing


    
## Reinforcement Learning Introduction

Reinforcement Learning is a Machine Learning paradigm that takes inspiration from how animals learn to interact with their environment in a cyclic process of trial and error.

The learning entity is called agent and at each turn (or instant, in case of continuous time) can observe its surrounding environment and has to decide what to do based on this observation and its past expercience. 

Usually the agent has a task to accomplish (e.g. collecting a resource or winning a match) that can be measured with some scoring function; the score of the whole episode (match) is usually called return. After every action, the agent will receive a feedback, called reward, on how its action has influenced the return; in other words the return will be the sum of all the rewards obtained during the episode.

The main problem for the agent is that it has to estimate the best action to take based on information that it doesn't have, i.e. the result that its actions will have in the future. Hence it needs to play many times the game to collect enough information to learn from past experiences.

Most RL algorithms tackle this problem learning to estimate either the value of each move (action) in each situation (state) or a behavioural policy; in each case they will provide an estimate of which is the best action to take in a given state.

<img src="Support_material/reinforcement-learning-fig.jpg">

## A simple algorithm: Q-value with tabular methods

**Main idea:** if we were able to store in a matrix (look-up table, hence tabular methods) `Q(state,action)` the returns that we will get choosing a given action `a` in a given state `s` for each state and action, when we will need to choose what to do in an observed state `s'` we will just have to look all the entries `Q(s',*)` and select the action corresponding to the greatest entry. The values stored in the matrix Q will be called Q-values (that should stand for quality values).

### But how we can estimate these Q-values?

Imagine that the task of our agent is to navigate in a 2D grid from one point A to another point C in the fastest possible way: it starts from point A, then goes right to point B and finally (to make it simple) down to point C. 
Then the return for this task will be the one for choosing right when it was in A and to choosing down when it was in B: `Q(A,right) = r(A,right) + r(B,down)`.
But for B the Q-value will be exactly the reward, because it's a single move game from its perspective: `Q(B,down) = r(B,down)`. <br>
Hence in the end we have:

`Q(A,right) = r(A,right) + Q(B,down)`

provided the fact that in state B going down is the best action!

In general we can redefine the **Q-value** of a state-action pair `(s,a)` as the **reward obtained** `r(s,a)` for taking that action in that state **plus** the **return** that we will get taking the **best possible action in the new state** `s'`, that is estimated as `max[Q(s',a')]`.

Following this idea in the end we will:
* start initializing all Q-values to the same value (0 for example);
* update $Q(s,a) = (1-\alpha)Q(s,a) + \alpha [r(s,a) + max_{a'}Q(s',a')]$, where $\alpha$ is called learning rate and controls the trade-off between learning speed and algorithmic stability.




## How much experience do we have to do?

The main problem with tabular method is that we do not implement a notion of similarity between states, but we process information in a way that **past experiences similar but not identical between themselves are completely uninformative of each other**. This means that if we want to learn the optimal policy in each single situation we need to experience all those situations. We define complexity of the state-action space the combination of all possible state for all possible actions.

In the following sessions we will analyse the state-action complexity of the Halite environment for a single agent and show that it is too complex to deal with without approximations. Then we will show how it is possible to encode the information contained in the state variable (with some loss of information) in such a way that the experience needed to learn the environment instead of scaling exponentially in the map area becomes linear in it.

## State complexity and state approximation

To describe the state of a single ship system we have to specify:
- one of $(map\_size)^2$ possible positions for the ship ($49$ in the $7 \times\ 7$ case, up to $64^2 = 4096$ for the largest map);
- one of $h\_lev = 1000$ possible values of halite for each cell in the map;
- one of $h\_lev = 1000$ possible values of carried halite.

<img src="Support_material/single-agent-state-space.png">

The state of the system is defined by the position of the ship + the halite in EACH cell + the halite carried by the ship. We can have almost all the possible combinations of the values that those variables can assume, thus we have the combinations of $1000$ values of halite for $50$ cells ($49$ of the map + the one carried by the ship) all multiplied by $49$ possible positions of the ship, for a total of $49\times (10^3)^{50} = 49\times 10^{150}$ possible states. If instead we consider the largest map of $64 \times 64$ we arrive at $4096\times 10^{12288}$ possible states.

The general formula can be written as: 
$$\# states = (map\_size)^2 \times (h\_lev)^{map\_size^2+1}$$

where we consider the map always centered on the shipyard, hence in this framework its position is fixed and not considered as a variable.
Of course this state representation is totally uncontrollable, because it scales exponentially in the number of cells of the map.

To tackle this issue we choose to drastically reduce the amount of information that is observed by the ship through two processes: partial observation and state abstraction.



### Partial observation: depth of field

The most expensive dependence in the formula about the number of states of the system is the exponent at which is elevated $h\_lev$. This is obtained considering all possible combinations of halite for all the cells of the map and the halite carried by the ship. A different approach is to consider only the halite inside the field of view of the agent and restrict the depth of field to the minimal possible quantity, i.e. nearest neighbors. In this way, independently from the $map\_size$ we get an exponent that in 2D is equal to 6 (4 for the neighbors, 1 for the state in which the ship stands and 1 for the halite it carries).

In other words, we get: 

$$\# states = (map\_size)^2 \times (h\_lev)^{6}$$

that yields $4.9 \times 10^{19}$ for the $7 \times 7$ map, that is still not manageable, but considerably smaller ( of order $ \approx 10^{130}$).

### State abstraction: halite quantization

Differently from the restriction on the observation space, that are somewhat straightforward, the state abstraction must contain some hypothesis about the environment that involve knowing the model of the environment. For example, if we were in the situation of not knowing how the halite is collected we probably would have done a different choice.
Since we know that the ship collects an amount of halite proportional to the halite in the cell (25% of it, to be more exact) and pays a fee of 10% of the halite contained in a cell to leave it, we are more interested in having encoded the notions of "low" and "high" halite levels, instead of sampling with precision the middle-high half of the halite scale. 
To be more specific, we choose to approximate the information about the halite using for $h_{lev} = 3$ halite levels and the following encoding:
- $h = 0$ if $halite \le 10$; 
- $h = 1$ if $10 < halite \le 100$; 
- $h = 2$ if $100 < halite \le 1000$.

The important part is that the halite is quantized in intervals that grow of a decade each, but we could also test adding a fourth level.
In this way the number of states of the system becomes:

$$\# states = (map\_size)^2 \times 3^{6}$$

yielding for a $7 \times 7$ map $35.721$ states, that is reachable with our resources. 

### State abstraction: meta-informations

The problem of these manipulations is that now the ship has access only to local informations and lacks the knowledge about the position of the shipyard (but again, being the latter fixed, only the ship position is needed) and that of distant halite deposits. In order to enhance the ability of the ship to find those deposits, we encode in a 4-states variable the direction that the ship should take to go towards the nearest and richest deposit.

<img src="Support_material/high-level-features.png">


### Final state complexity

In this final formulation, the total number of states that needs to be experienced by the ship is:

$$\# states = (map\_size)^2 \times 3^{6} \times  4$$

yielding for a $7 \times 7$ map the final result of $142.884$ states. Considering that each of these states has to be multiplied for all the possible actions (5) and requires 64 bits, i.e. 8 bytes, to be stored, the memory required to store the Q-value table is $1.143.072 \times 5$ bytes  = 5.7 Mb.

<img src="Support_material/partial-observability.png">

## Take-home messages

1. Q-value is a simple and intuitive algorithm to be learned by reinforcement;
2. Tabular methods are simple to implement, but do not scale well with state-action complexity;
3. By restricting the depth of field of the agent, quantizing the halite levels and providing some high-level meta informations, we reduce drastically the complexity of the problem, at the cost of loosing some information.